In [1]:
import pymysql
import requests
import pandas as pd
import numpy as np

### Genera un df con el archivo csv "TRANSFERS"

In [3]:
df = pd.read_csv('C:\\Users\\cfari\OneDrive\\Escritorio\\PF_GRUPAL\\DATASET\\TRANSFERS.csv', encoding='UTF-8', delimiter=',', decimal='.')
df.head(3)

,row_id,subject_id,hadm_id,icustay_id,dbsource,eventtype,prev_careunit,curr_careunit,prev_wardid,curr_wardid,intime,outtime,los
0,54440,10006,142345,206504.0,carevue,admit,NaN,MICU,NaN,52.0,2164-10-23 21:10:15,2164-10-25 12:21:07,39.18
1,54441,10006,142345,NaN,carevue,transfer,MICU,NaN,52.0,45.0,2164-10-25 12:21:07,2164-11-01 17:14:27,172.89
2,54442,10006,142345,NaN,carevue,discharge,NaN,NaN,45.0,NaN,2164-11-01 17:14:27,NaN,NaN


In [4]:
df = df.replace(np.nan, 0)

### Se conecta con la BD MySQL

In [ ]:
con = pymysql.connect(
      host='database-proyecto-final.cpjrl88bhdul.us-east-1.rds.amazonaws.com',
      database='databaseuci',
      user='admin',
      password='datascience'
)

### Inserta los registros del df "TRANSFERS" en la BD

In [5]:
sql_insert = ''' INSERT 
                   INTO  TRANSFERS
                         (row_id,subject_id,hadm_id,icustay_id,dbsource,eventtype,prev_careunit,curr_careunit,prev_wardid,curr_wardid,intime,outtime,los)
                 VALUES  (%s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s,%s) '''
cursor = con.cursor()
for i in range(0, len(df)):
    row = int(df.loc[i, 'row_id'])
    subj = int(df.loc[i, 'subject_id'])
    hadm = int(df.loc[i, 'hadm_id'])
    icu = int(df.loc[i, 'icustay_id'])
    dbs = df.loc[i, 'dbsource']
    ety = df.loc[i, 'eventtype']
    prc = df.loc[i, 'prev_careunit']
    cuc = df.loc[i, 'curr_careunit']
    prw = int(df.loc[i, 'prev_wardid'])
    cuw = int(df.loc[i, 'curr_wardid'])
    itm = df.loc[i, 'intime']
    otm = df.loc[i, 'outtime']
    los = int(df.loc[i, 'los'])
    reg_s = (row, subj, hadm, icu, dbs, ety, prc, cuc, prw, cuw, itm, otm, los)
    cursor.execute(sql_insert, reg_s)
#cursor.fetchone()

### Confirma la inserción de los registros a la BD

In [6]:
con.commit()

### Cierra el cursor y la conexión a la BD

In [7]:
cursor.close()
con.close()